# 初始化Lanchain
# 安装依赖
* 运行环境: python 3.11.8 , conda
* 开发环境: langchain v0.1.11 , openai v0.28.1
```sh 
conda install langchain -c conda-forge
conda install openai==v0.28.1 -c conda-forge
conda install Chroma
```

In [ ]:
# RUN apt-get update -y && apt-get install libmagic1 libmagic-dev -y
%pip install python-magic-bin==0.4.14
%pip install python-libmagic
%pip install unstructured
%pip install bilibili-api

In [ ]:
%pip install langchain
%pip install langchain-openai
%pip install Chroma
%pip install -U duckduckgo-search
%pip install faiss-cpu
%pip install huggingface-hub
%pip install -U langchain-community faiss-cpu langchain-openai tiktoken
%pip install sentence-transformers


In [3]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (ChatPromptTemplate,PromptTemplate,SystemMessagePromptTemplate,
                               AIMessagePromptTemplate,HumanMessagePromptTemplate,)
from langchain.schema import ( HumanMessage, SystemMessage, AIMessage )
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import UnstructuredMarkdownLoader

import os

os.environ['OPENAI_API_BASE'] = 'https://openai.api2d.net/v1'
os.environ['OPENAI_API_KEY'] = 'fk224285-rmfTvLRKQyarQdyF5YBX6IlfKVKTj1y1'

chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=1500)


In [4]:

response = chat.predict_messages([
  SystemMessage(content="你是一个对人工智能一无所知的聊天机器人。当你被问及人工智能时，你必须说“我不知道”"),
  HumanMessage(content="你知道什么是AI吗?")
])
print(response)

content='我不知道' additional_kwargs={} example=False


In [14]:
system_template="You are a professional translator that translates {src_lang} to {dst_lang}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

message = chat_prompt.format_prompt(
    src_lang="English",
    dst_lang="Chinese", 
    user_input="Did you eat in this morning?"
).to_messages()

response = chat.predict_messages(message)
print(response)

content='你今天早上吃了吗？' additional_kwargs={} example=False


In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import OpenAI

prompt = PromptTemplate(
    input_variables=["color"],
    template="What is the hex code of color {color}?",
)
chain = LLMChain(llm=chat, prompt=prompt)

print(chain.run("green"))
print(chain.run("red"))
print(chain.run("yellow"))


The hex code for the color green is #008000.
The hex code for the color red is #FF0000.
The hex code for the color yellow is #FFFF00.


In [ ]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory( k=1)
memory.save_context({"input": "Hi, LangChain!"}, {"output": "Hey!"})
memory.save_context({"input": "Where are you?"}, {"output": "By your side"})

memory.chat_memory.messages


[HumanMessage(content='Hi, LangChain!', additional_kwargs={}, example=False),
 AIMessage(content='Hey!', additional_kwargs={}, example=False),
 HumanMessage(content='Where are you?', additional_kwargs={}, example=False),
 AIMessage(content='By your side', additional_kwargs={}, example=False)]

In [2]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
import os

os.environ['http_proxy'] = "socks5h://127.0.0.1:7890" 
os.environ['https_proxy'] = "socks5h://127.0.0.1:7890" 

tools = load_tools(["ddg-search", "llm-math"], llm=chat)
agent = initialize_agent(tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("中国尊和哈利法塔?它们相差多高? 它们谁最高? 使用中文回答,并返回数值")




> Entering new AgentExecutor chain...
我需要查一下中国尊和哈利法塔的高度，然后比较它们的差距。
Action: duckduckgo_search
Action Input: 中国尊高度
Observation: 本页面列举了中国大陆目前最高的10座高楼，包括上海中心大厦、平安金融中心、高银金融117、广州塔等，以及其他港澳台地区的两座高楼。每座高楼都有其高度、城市、建成年份、设计特点和观光厅等信息，供高楼爱好者和建筑爱好者参考。 上海中心大厦 标准高度： 632 米 (2073 英尺) 屋顶高度： 587 米 楼层数： 128 观光厅高度： 552 米 用途： 办公，酒店，观光，书店，博物馆，零售 建成年份： 2015 所在城市： 上海 中国尊高度528米，地上108层，地下7层。 其自下而上的菱形机理，源于中国传统器皿之一——竹器。 中国人爱竹，竹比作君子，淡雅有气节，自古有"不可居无竹"之语。 建筑顶部的空间设计取自"孔明灯"的形态意向，呈冉冉上升之态。 在柔美而赋有寓意的外形设计下，技术的更新运用也让"中国尊"有了更多亮点，取得了多个世界第一。 智能顶升钢平台、永临结合消防系统、BIM技术全周期应用、完善的消防系统、抗震能力、低碳环保的设计贯穿于中国尊各个细节中，通过调节建筑物的高度结构和方位朝向，充分利用冬季阳光和夏季自然风的流动；建立起一套区域能源系统，实现区域集中供冷供热系统，减少20%-40%的能源消耗。 此外，中国尊使用的电梯是世界首创的服务高度超500米跃层电梯。 🎉答案是：目前，中国尊作为商务办公大楼，并未向公众开放观光服务，主要接待的是内部工作人员。 也就是说，暂时无法像游览其他城市观景台那样，登上中国尊顶端饱览京城美景。 该大厦坐落在北京市商务中心区的核心地带，东起金和东路，南接规划中的绿地，西至金和路，北靠光华路。 🌈尽管如此，"中国尊"的建筑设计和工程成就仍然值得我们一探究竟。 其设计理念源自中国古代礼器"尊"，寓意着庄重与崇高。 建筑外形自下而上逐渐收紧再放大，形如古代酒器"樽"，同时融入了竹编、孔明灯以及城门等中国传统元素，巧妙地将东方神韵与现代摩天大楼相结合。 💡值得一提的是，"中国尊"在工程技术方面创下了多项世界之最和国内纪录。 三、总结 北京中国尊的结构与力学之美不仅仅体现在其外观上，

'300 米'

In [12]:

import os
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma,FAISS,VectorStore
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.llm import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT,QA_PROMPT
from IPython.display import Markdown, display
from datetime import date
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.chains.summarize import load_summarize_chain
from lang_funcs import (load_docs_by_extension,split_docs,
        create_vectorstore_faiss,create_vectorstore,
        load_openai_embeddings,load_huggingface_embeddings,
        create_knowledge_chain,create_summarize_chain,create_llm_openai
)


chat = create_llm_openai( apikey='fk224285-rmfTvLRKQyarQdyF5YBX6IlfKVKTj1y1',
                         apibase='https://openai.api2d.net/v1')

#embeddings = load_openai_embeddings()
embeddings = load_huggingface_embeddings('all-MiniLM-L6-v2')

doc_url = "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-9-2023"

documents = load_docs_by_extension(doc_url)
#documents = load_docs_by_extension('./README.md')

print (f'There are {len(documents)} document(s) in this document.')
print (f'There are {len(documents[0].page_content)} characters in the first page of your document.')
 
#vectorstore = create_vectorstore(documents,embeddings)
vectorstore = create_vectorstore_faiss(documents,embeddings,"russian-offensive-campaign")

Q = create_knowledge_chain(chat,vectorstore.as_retriever())

result =  Q({"question":"请详细讲一下这个文档主要内容是什么?"})["answer"].strip()

display(Markdown(f"<p>{result}</p>"))


chain = create_summarize_chain(chat)
summ = chain({"input_documents": documents}, return_only_outputs=True)

print(summ['output_text'])



There are 1 document(s) in this document.
There are 28164 characters in the first page of your document.
Creating the embeddings using FAISS


<p>这个文档主要内容是关于俄罗斯对乌克兰的进攻行动的评估报告。报告包括了关于瓦格纳集团停止从监狱招募士兵、克里姆林宫为乌克兰战争准备的措施、一位与瓦格纳集团有关的俄罗斯军事博主呼吁解雇俄罗斯国防部长谢尔盖·绍伊古、以及克里姆林宫不愿意遏制极端民族主义亲战争人物的言论等内容。此外，报告还包括了俄罗斯在乌克兰东部的主要进攻行动、支持行动、动员和力量生成努力、以及在俄罗斯占领区域的活动等方面的详细描述。</p>

2023年2月9日的俄罗斯进攻战役评估提供了有关乌克兰冲突各个方面的更新。关键点包括瓦格纳集团的招募实践，俄罗斯努力为一场持久战争做好国防工业准备，要求解雇俄罗斯国防部长，以及在乌克兰东部继续进行攻势行动。报告还强调了俄罗斯在不同地区的军事活动，包括卢甘斯克、顿涅茨克和扎波罗热州，以及俄罗斯占领领土和白俄罗斯的发展。评估未涵盖俄罗斯的战争罪行，但谴责这些行为。该报告基于公开可获得的信息和来源。


In [ ]:
from langchain.agents import tool
from langchain.agents import load_tools
from langchain.agents import AgentType,initialize_agent
from lang_tools import *

tools = load_tools(["ddg-search", "llm-math"], llm=chat)

agent1 = initialize_agent(
    tools + [time,weather,note],
    llm=chat,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True)

result =  agent1.invoke("打开记录本")
print(result)

result =  agent1.invoke("今天是几号?")
print(result)
result =  agent1.invoke("今天上海的天气怎么样?")
print(result)

